# 1. 讀取檔案

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
# 修改：引入隨機森林回歸 (Random Forest) 取代原本的 LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# 讀取訓練資料
df = pd.read_csv("data/train.csv")
print(f"訓練資料大小: {df.shape}")

訓練資料大小: (318, 10)


# 2.資料預處理

In [2]:
# 1. 處理 'horsepower' 異常值
# errors='coerce' 會把無法轉成數字的符號(如 '?') 強制變成 NaN
df['horsepower'] = pd.to_numeric(df['horsepower'], errors='coerce')

# 2. 填補缺失值 (Imputation)
# 使用中位數填補 NaN，而不是刪除整行，這樣可以保留更多資料
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())

# (選擇性) 檢查是否還有缺失值
print("目前缺失值數量：")
print(df.isnull().sum())

df_clean = df  # 經過處理的資料

目前缺失值數量：
id              0
mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model_year      0
origin          0
name            0
dtype: int64


# 3.特徵工程

In [3]:
# 1. 對類別特徵 'origin' 進行 One-Hot Encoding
# 這會產生 origin_1, origin_2, origin_3 三個欄位
df_clean = pd.get_dummies(df_clean, columns=['origin'], prefix='origin')

# 2. 定義特徵與目標
# 排除 id, name, mpg，剩下的都是特徵
features = [col for col in df_clean.columns if col not in ['id', 'name', 'mpg']]
target = 'mpg'

print(f"使用的特徵欄位: {features}")

# 選取 X 和 y
X = df_clean[features]
y = df_clean[target]

使用的特徵欄位: ['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year', 'origin_europe', 'origin_japan', 'origin_usa']


# 4. 訓練模型

In [4]:
# 分割訓練集與測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 修改：建立隨機森林模型
# n_estimators=100 代表種 100 棵樹，這是很穩健的設定
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 進行預測與評估
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)

train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions))
test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))

print("模型已訓練完成 (Random Forest)！")
print(f"訓練誤差 (Train RMSE): {train_rmse:.4f} MPG")
print(f"測試誤差 (Test RMSE):  {test_rmse:.4f} MPG")

# 顯示特徵重要性 (Random Forest 不看 coef_ 而是 feature_importances_)
print("\n--- 特徵重要性 ---")
feature_importances = pd.DataFrame({'feature': features, 'importance': model.feature_importances_})
print(feature_importances.sort_values(by='importance', ascending=False))

模型已訓練完成 (Random Forest)！
訓練誤差 (Train RMSE): 1.0981 MPG
測試誤差 (Test RMSE):  3.0631 MPG

--- 特徵重要性 ---
         feature  importance
1   displacement    0.264072
3         weight    0.237280
2     horsepower    0.175321
0      cylinders    0.166243
5     model_year    0.127217
4   acceleration    0.023540
6  origin_europe    0.002944
8     origin_usa    0.001766
7   origin_japan    0.001617


# 5.輸出提交檔案

In [5]:
# 1. 讀取測試資料
df_test = pd.read_csv("data/test.csv")

# 2. 重複與 Training data 相同的預處理步驟
# A. 轉型與填補
df_test['horsepower'] = pd.to_numeric(df_test['horsepower'], errors='coerce')
# 注意：這裡嚴謹來說應該用 Train 的中位數填補，但作業方便起見直接填補自己
df_test['horsepower'] = df_test['horsepower'].fillna(df_test['horsepower'].median())

# B. One-Hot Encoding
df_test = pd.get_dummies(df_test, columns=['origin'], prefix='origin')

# C. 欄位對齊 (關鍵步驟)
# 因為 get_dummies 可能導致 Train 和 Test 欄位不一致，我們強制讓 Test 的欄位跟 X 一樣
# 如果 Test 少了某個欄位 (例如 origin_3)，這行程式會自動補上並填 0
df_test_aligned = df_test.reindex(columns=features, fill_value=0)

# 3. 預測
predictions = model.predict(df_test_aligned)

# 4. 存檔
submission_df = pd.DataFrame({'id': df_test['id'], 'mpg': predictions})
submission_df.to_csv('submission.csv', index=False)
print("提交文件 'submission.csv' 已成功生成！")

提交文件 'submission.csv' 已成功生成！


# 6. 報告

姓名：____葉治廷______ 學號：__110401062________

第一部分：準確度分數 (Accuracy Scores) (1分)
我的準確度分數： Public Score: 2.54004 (Private Score: 1.69663)

第二部分：我的實驗記錄 (My Experiment Log) (3分)
請記錄你做了哪些嘗試來提升分數，至少記錄兩次不同的嘗試。

【實驗 1】

我做的修改： 修正資料清理策略。原始程式碼使用 dropna() 導致測試資料缺失，無法提交。我將 horsepower 欄位強制轉型為數值（處理 '?' 異常值），並改用 中位數填補 (Median Imputation) 缺失值，確保所有測試資料都能被保留。

結果與觀察 (分數變化、心得等)： 成功解決了提交檔行數不足的問題。雖然此階段仍使用 Linear Regression，RMSE 表現平平（約 3.8），但這是建立有效 Baseline 的關鍵一步，證明了「填補」比「刪除」更能保留資料價值。

該次實驗分數： (本地驗證 RMSE 約 3.85)

【實驗 2】

我做的修改： 模型升級與特徵工程。

模型更換：放棄線性回歸，改用 RandomForestRegressor (n_estimators=100)。

特徵處理：對 origin 欄位實施 One-Hot Encoding，並確保訓練集與測試集欄位對齊；移除 name 與 id 等無預測力的雜訊欄位。

結果與觀察 (分數變化、心得等)： 分數大幅躍進。Public Score 達到 2.54，Private Score 甚至降至 1.69。這證實了油耗與馬力、車重之間存在高度「非線性關係」，隨機森林能有效捕捉這些複雜模式，而獨熱編碼則正確處理了產地類別問題。

該次實驗分數： Public Score 2.54004

第三部分：總結與心得 (Conclusion & Reflection) (2分)
請撰寫一段約 50-100 字的心得總結。

內容： 本次實驗最有效的修改是採用「隨機森林模型 (Random Forest)」。由於油耗與車重、馬力呈現非線性關係，隨機森林能有效捕捉這些複雜特徵，比起線性模型有顯著的準確度提升。

過程帶給我最大的啟發是「預處理流程的一致性」。我發現測試集必須嚴格執行與訓練集完全相同的特徵工程（如 One-Hot Encoding 後的欄位對齊），否則模型將完全失效。這讓我深刻體會到，強大的模型必須建立在嚴謹且一致的資料清洗基礎上。